## RANDOM TESTS

In [2]:
import numpy as np
import pandas as pd
import rdflib as rdf
import requests
import json
import re
import sys
import SPARQLWrapper as sw
from PyMovieDb import IMDB

In [3]:
WIKIDATA_URL = "https://query.wikidata.org/sparql"

In [4]:
def get_query_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = sw.SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(sw.JSON)
    return sparql.query().convert()

In [36]:
anime_name_query = "attack"

# get the anime id from wikidata
query = """
SELECT ?anime ?animeLabel ?animeIMDBid ?default_review_score WHERE {
  ?anime wdt:P31/wdt:P279* wd:Q1107 .
  ?anime wdt:P345 ?animeIMDBid .
  ?anime rdfs:label ?animeLabel.

  # stuff that may or not be there
  OPTIONAL{?anime wdt:P444 ?default_review_score}
  
  # filters
  FILTER(LANG(?animeLabel) = "en")
  FILTER(REGEX(?animeLabel, "%s", "i"))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
} ORDER BY ?animeLabel
""" % anime_name_query

results = get_query_results(WIKIDATA_URL, query)

print(json.dumps(results, indent=2, sort_keys=True))


{
  "head": {
    "vars": [
      "anime",
      "animeLabel",
      "animeIMDBid",
      "default_review_score"
    ]
  },
  "results": {
    "bindings": [
      {
        "anime": {
          "type": "uri",
          "value": "http://www.wikidata.org/entity/Q98915343"
        },
        "animeIMDBid": {
          "type": "literal",
          "value": "tt0296304"
        },
        "animeLabel": {
          "type": "literal",
          "value": "Attack No. 1",
          "xml:lang": "en"
        },
        "default_review_score": {
          "type": "literal",
          "value": "6.9/10"
        }
      },
      {
        "anime": {
          "type": "uri",
          "value": "http://www.wikidata.org/entity/Q22126305"
        },
        "animeIMDBid": {
          "type": "literal",
          "value": "tt2560140"
        },
        "animeLabel": {
          "type": "literal",
          "value": "Attack on Titan",
          "xml:lang": "en"
        }
      },
      {
        "anime": {
 

In [38]:
# get imdb score for Neon Genesis Evangelion
# entity Q662
# imdb id tt0112159
imdb = IMDB()
imdb_score = json.loads(imdb.get_by_id("tt2560140"))["rating"]["ratingValue"]
print(imdb_score)

9.1
